# Control Variate Technique for Binomial Tree Prices

### Motivation
This section illustrates the use of the Control Variate technique for reducing the variance of an estimate for an option price calculated on a Binomial tree. See  https://en.wikipedia.org/wiki/Control_variates or Simulation by S.M. Ross. 
Here we price an American option by using the corresponding European option which has an analytic expression for it's price given by the Black-Scholes formula. We first estimate the optimal coefficient, $c^*$ in the expression
$$Z\equiv X + c(Y - \mu_Y)$$ so as to minimize it's variance: 
$$Var[Z] = Var[X] + c^2Var[Y] +2cCov[X,Y]$$
$$c^* = -Cov(X,Y) / Var(Y).$$ We then have $$E[Z] = E[X]$$ $$Var[Z] = Var[X] -(Cov[X,Y])^2 / Var[Y]$$ 
or:
$${Var[Z]\over Var[X]} = (1 - Corr^2[X, Y])$$
By choosing X, Y as the price of the American, European option calculated on the binomial tree, and setting $\mu_Y$ as the Black-Scholes price of the European option, $Z$ calculated from the binomial tree will estimate the price of the American option with reduced variance. 

This method was discussed by Hull \& White The Use of the Control Variate Technique in Option Pricing, J o Finance, 42 (June 1987), 281-300, but they used c = 1. THE CONVERGENCE OF BINOMIAL TREES FOR PRICING
THE AMERICAN PUT M.Joshi 2008 also uses c=1. The improvement using optimal $c=c^*$ was discussed here: https://scholars.lib.ntu.edu.tw/bitstream/123456789/165629/1/22.pdf

What variance? Where's the randomness in all of this? Usually this technique will be applied, for example, to estimating option prices with a Monte Carlo simulator as the sample mean of many prices simulated by MC. The optimal $c^*$ can then be estimated from the sample estimates of $Cov[X,Y], Var[Y]$ obtained from those prices. $Z$ will then exhibit reduced variance and produce more accurate estimates. Here we let the number of time steps used in the construction of the binomial tree play the role of randomness, i.e., we estimate $c^*$, $Cov[X,Y], Var[Y]$ from a 'sample' of trees having some range of `N_steps`. We estimate $c^*$ on a range of smaller trees($c^*_{small}$), then calculate $Z$ on large tree(s). Even if the true $c^*$ varies with tree size, $E[Z] = E[X]$ is always true, and hopefully we get reduced variance on the larger trees using $c^*_{small}$ instead of the computationally more expensive $c^*_{large}$.




In [1]:
// option_pricing.hpp:
#include <functional>
#include <iostream>

namespace OptionPricing {

    class Asset {
    public:
        Asset(double S_o, double q, double sigma);
        Asset(const Asset&) = default;

        double   S_o() const { return _S_o; }
        double     q() const { return _q; }
        double sigma() const { return _sigma; }

    private:
        double _S_o;
        double _q;
        double _sigma;
    };


    class Terms {
    public:
        enum class Style { Euro, Amer, AsianPrc, AsianStrike, AsianExotic };
        enum class Type { Call, Put, Other };
        enum class AvgType { Arith, Geom, Other, None };

        static std::function<void(int, double, double&)> arith_path_accum_fn;
        static std::function<void(int, double, double&)> geom_path_accum_fn;

        Terms(Style, Type, double T, double K, AvgType = AvgType::Arith);
        Terms(Style, Type, double T, AvgType = AvgType::Arith);
        Terms(Style, Type, double T, std::function<double(double)> payoff);
        Terms(Style, Type, double T, std::function<double(double, double)> path_payoff,
            std::function<void(int, double, double&)> path_accum, double init_accum);
        Terms(const Terms&) = default;
        Terms() = delete;

        Style style() const { return _style; }
        Type type() const { return _type; }
        double K() const { return _K; }
        double T() const { return _T; }
        AvgType avgType() const { return _avg_type; }
        std::function<double(double)> payoff_fn() const { return _payoff; }
        std::function<double(double, double)> path_payoff() const { return _path_payoff; }
        std::function<void(int, double, double&)> path_accum() const { return _path_accum_fn; }
        double init_accum() const { return _init_accum; }

    private:
        const Style _style;
        const Type _type;
        const double _T;
        const double _K;
        AvgType _avg_type;
        std::function<double(double)> _payoff{};
        std::function<double(double, double)> _path_payoff{};
        std::function<void(int, double, double&)> _path_accum_fn{};
        double _init_accum{ 0.0 };
    };


    class Option {
    public:
        Option(const Terms&, const Asset&);

        double btree_prc(int N_steps, double r, bool get_greeks);
        double MC_prc(int N_sims, int N_steps, double r, bool get_greeks,
            bool antithetic, double& sd_prc);


        struct Greeks {
            double delta{ 0 };
            double gamma{ 0 };
            double theta{ 0 };
            double   rho{ 0 };
            double  vega{ 0 };
        };

        Terms  terms() const { return _terms; }
        Asset  asset() const { return _asset; }
        double price() const { return _prc; }
        Greeks all_greeks() const { return _greeks; }
        double delta() const { return _greeks.delta; }
        double gamma() const { return _greeks.gamma; }
        double theta() const { return _greeks.theta; }
        double   rho() const { return _greeks.rho; }
        double  vega() const { return _greeks.vega; }

    private:
        const Terms _terms;
        const Asset _asset;
        double _prc;
        Greeks _greeks;
    };

    double btree_prc_opt(Option&, int N_steps, double r, bool greeks);

    double btree_prc_terms_asset(const Terms&, const Asset&, int N_steps, double r,
        Option::Greeks*); // Note user danger here.

    double btree_prc_raw(int N_steps, double r,
        Terms::Style style, double T, std::function<double(double)>,
        double S_o, double q, double sigma, Option::Greeks*);


    double MC_prc_opt(Option&, int N_sims, int N_steps, double r, bool greeks,
        bool antithetic, double& sd_prc);

    double MC_prc_terms_asset(const Terms&, const Asset&, int N_sims, int N_steps, double r,
        Option::Greeks*, bool antithetic, double& sd_prc);

    double MC_prc_raw(int N_sims, int N_steps, double r,
        Terms::Style style, double T, std::function<double(double)> payoff,
        std::function<double(double, double)> path_payoff,
        std::function<void(int, double, double&)> path_accum, double init_accum,
        double S_o, double q, double sigma, Option::Greeks*,
        bool antithetic, double& sd_prc);


    std::ostream& operator<< (std::ostream&, const Asset&);
    std::ostream& operator<< (std::ostream&, const Terms&);
    std::ostream& operator<< (std::ostream&, const Option&);

}

In [2]:
// option_pricing.cc:
// #include "option_pricing.hpp"
// #include "fexcpt.hpp"
#include <algorithm>
#include <cmath>
#include <limits>
#include <iostream>
#include <random>


namespace OptionPricing {
    using std::exp;
    using std::sqrt;
    
    Asset::Asset(double S_o, double q, double sigma)
        : _S_o{ S_o }, _q{ q }, _sigma{ sigma } { }


    Terms::Terms(Style s, Type t, double T, double K, AvgType avg)
        // Set terms of Euro, Amer, or AsianPrc Call or Put, each of which define their 
        // _payoff(S) in terms of the strike K in the usual vanilla way.
        try : _style{ s }, _type{ t }, _T{ T }, _K{ K }, _avg_type{ avg } {
        if (t == Type::Other || s == Style::AsianStrike || s == Style::AsianExotic
            || avg == AvgType::Other || _K < 0 || _T <= 0) {
            throw std::invalid_argument(
                "Must provide Euro, Amer or AsianPrc Call or Put with valid K and T.");
        }
        if (s == Style::AsianPrc && avg != AvgType::Arith && avg != AvgType::Geom)
            throw std::invalid_argument("AsianPrc must do an Arith or Geom path price average.\n \
                                         -Use AsianExotic to specify Other path price avgeraging.");
        if (_type == Type::Call)
            _payoff = [K](double S) { return std::max(S - K, 0.0); };
        else
            _payoff = [K](double S) { return std::max(K - S, 0.0); };
        if (_style == Style::AsianPrc) {
            if (_avg_type == AvgType::Arith) {
                _path_accum_fn = arith_path_accum_fn;
                _init_accum = 0.0;
            } else {
                _path_accum_fn = geom_path_accum_fn;
                _init_accum = 1.0;
            }
        } else  // Euro and Amer have no path averaging:
            _avg_type = AvgType::None;
    }
    catch (std::exception& e) {
        std::cout << "In Terms's ctor1: " << e.what() << std::endl;
        throw;
    }

    Terms::Terms(Style s, Type t, double T, AvgType avg)
        // Set terms of AsianStrike. Like AsianPrc, must be a Call or Put and uses only Arith or 
        // Geom path price averaging, -Use AsianExotic to specify other path averaging.
        try : _style{ s }, _type{ t }, _T{ T }, _K{ -1 }, _avg_type{ avg } {
        if (s != Style::AsianStrike || t == Type::Other ||
            (avg != AvgType::Arith && avg != AvgType::Geom) || _T <= 0) {
            throw std::invalid_argument("Must provide AsianStrike Call or Put with valid T\n \
             and can only do Arith or Geom path price average:\n \
              -Use AsianExotic to specify Other path price averaging.");
        }
        if (_type == Type::Call)
            _path_payoff = [](double K_avg, double S) { return std::max(S - K_avg, 0.0); };
        else
            _path_payoff = [](double K_avg, double S) { return std::max(K_avg - S, 0.0); };

        if (_avg_type == AvgType::Arith) {
            _path_accum_fn = arith_path_accum_fn;
            _init_accum = 0.0;
        } else {
            _path_accum_fn = geom_path_accum_fn;
            _init_accum = 1.0;
        }
    }
    catch (std::exception& e) {
        std::cout << "In Terms's ctor2: " << e.what() << std::endl;
        throw;
    }

    Terms::Terms(Style s, Type t, double T, std::function<double(double)> payoff)
        // Set terms of Euro or Amer option that is not Call or Put (i.e. is Other)
        // User provides the non-vanilla payoff function.
        try : _style{ s }, _type{ t }, _T{ T }, _K{ -1 }, _avg_type{ AvgType::None } {
        if ((s != Style::Amer && s != Style::Euro) || t != Type::Other || _T <= 0) {
            throw std::invalid_argument("Must provide Euro or Amer with valid T\n \
             and provide payoff function of this exotic(Type::Other) Terms object.");
        }
        _payoff = payoff;
    }
    catch (std::exception& e) {
        std::cout << "In Terms's ctor3: " << e.what() << std::endl;
        throw;
    }

    Terms::Terms(Style s, Type t, double T, std::function<double(double, double)> path_payoff,
        std::function<void(int, double, double&)> path_accum_fn, double init_accum)
        // Set terms of AsianExotic option. 
        // Both path_payoff and path_accum_fn (along with init_accum) need to be specified.
        try : _style{ s }, _type{ t }, _T{ T }, _K{ -1 }, _avg_type{ AvgType::Other } {
        if (s != Style::AsianExotic || t != Type::Other || _T <= 0) {
            throw std::invalid_argument("Must provide AsianExotic with valid T\n \
             path_payoff function of this exotic(Type::Other) Terms object.");
        }
        _path_payoff = path_payoff;
        _path_accum_fn = path_accum_fn;
        _init_accum = init_accum;
    }
    catch (std::exception& e) {
        std::cout << "In Terms's ctor4: " << e.what() << std::endl;
        throw;
    }


    std::function<void(int, double, double&)>
        Terms::arith_path_accum_fn { [](int j, double s, double& cum) {
        cum += (s - cum) / j;
    } };
    std::function<void(int, double, double&)>
        Terms::geom_path_accum_fn = [](int j, double s, double& cum) {
        cum *= pow(s / cum, 1.0 / j);
    };

    Option::Option(const Terms& t, const Asset& a)
        : _terms{ t }, _asset{ a }, _prc{ -1 }, _greeks{} {  }


    double Option::btree_prc(int N_steps, double r, bool get_greeks) {
        Greeks* g_ptr{ get_greeks ? &_greeks : nullptr };
        _prc = btree_prc_terms_asset(_terms, _asset, N_steps, r, g_ptr);
        return _prc;
    }

    double btree_prc_opt(Option& opt, int N_steps, double r, bool greeks) {
        return opt.btree_prc(N_steps, r, greeks);
    }

    double btree_prc_terms_asset(const Terms& t, const Asset& a, int N_steps, double r,
        Option::Greeks* g_ptr) {
        if (t.style() != Terms::Style::Amer && t.style() != Terms::Style::Euro) {
            throw std::invalid_argument("In btree_prc_raw, Bad Terms::Style.");
        }
        return btree_prc_raw(N_steps, r, t.style(), t.T(), t.payoff_fn(),
            a.S_o(), a.q(), a.sigma(), g_ptr);
    }

    double btree_prc_raw(int N, double r,
        Terms::Style style, double T, std::function<double(double)> payoff,
        double S_o, double q, double sigma, Option::Greeks* g_ptr) {

        if (style != Terms::Style::Amer && style != Terms::Style::Euro) {
            throw std::invalid_argument("In btree_prc_raw, Bad Terms::Style.");
        }

        double dt = T / N;
        double u = exp(sigma * sqrt(dt));
        double u2 = u * u;
        double d = 1. / u;
        double a = exp((r - q) * dt);
        double p = (a - d) / (u - d);
        double df = exp(-r * dt);

        struct Node {
            double s{ 0 };
            double f{ 0 };
        };
        int len_nodes = N + 1;
        Node* nodes = new Node[len_nodes]{};

        // Set s and f for final time step:
        nodes[0].s = S_o * std::pow(u, -N);
        nodes[0].f = payoff(nodes[0].s);
        for (int j = 1; j < N + 1; ++j) {
            if (j == N / 2) {  // Take advantage to tie node's s value to known value at middles:
                if (N % 2)     // Odd N, we're one step below middle:
                    nodes[j].s = S_o / u;
                else           // Even N, we're at the middle, so s == S_o:
                    nodes[j].s = S_o;
            } else {
                nodes[j].s = nodes[j - 1].s * u2;
            }
            nodes[j].f = payoff(nodes[j].s);
        }

        std::function<double(int)> step_back_Payoff;
        if (style == Terms::Style::Euro)
            step_back_Payoff = [p, df, nodes](int idx) {
            return df * (p * nodes[idx + 1].f + (1 - p) * nodes[idx].f); };
        else
            step_back_Payoff = [p, df, payoff, nodes](int idx) {
            return std::max(payoff(nodes[idx].s), df * (p * nodes[idx + 1].f + (1 - p) * nodes[idx].f)); };

        // Backstep from time i=N-1 to i=0, filling in nodes j=0 to j=i for each:
        double s1, s2, s3, s4, s5;  // Needed to save some values at i=1, 2 for greeks calculations.
        double f1, f2, f3, f4, f5;
        for (int i = N - 1; i >= 0; --i) {
            for (int j = 0; j <= i; ++j) {
                if (j == i / 2) {  // As above, tie to known values.
                    if (i % 2)     // Potential speed up: break j-loop into before and after middle..
                        nodes[j].s = S_o / u;
                    else
                        nodes[j].s = S_o;
                } else {
                    nodes[j].s = nodes[j].s * u;
                }
                nodes[j].f = step_back_Payoff(j);  // flag_Payoffe("Underflow for fine grid: Ok", false);
// see https://stackoverflow.com/questions/46611633/can-the-floating-point-status-flag-fe-underflow-set-when-the-result-is-not-sub-n
            }
            if (i <= 2) {  // Save some values to calculate delta, gamma, theta:
                if (i == 1) {
                    s1 = nodes[0].s;
                    s2 = nodes[1].s;
                    f1 = nodes[0].f;
                    f2 = nodes[1].f;
                }
                if (i == 2) {
                    s3 = nodes[0].s;
                    s4 = nodes[1].s;
                    s5 = nodes[2].s;
                    f3 = nodes[0].f;
                    f4 = nodes[1].f;
                    f5 = nodes[2].f;
                }
            }
        }

        double f0 = nodes[0].f;

        if (g_ptr) {
            g_ptr->delta = (f2 - f1) / (s2 - s1);
            g_ptr->gamma = 2 * ((f5 - f4) / (s5 - s4) - (f4 - f3) / (s4 - s3)) / (s5 - s3);
            g_ptr->theta = (f4 - f0) / (2 * dt);

            double incr = .0001;
            g_ptr->rho = (btree_prc_raw(N, r + incr, style, T, payoff, S_o, q, sigma, nullptr) - f0) / incr;
            g_ptr->vega = (btree_prc_raw(N, r, style, T, payoff, S_o, q, sigma + incr, nullptr) - f0) / incr;
            //g_ptr->delta2 = (btree_prc_raw(N, r, style, T, F, S_o + incr, q, sigma, nullptr) - f0) / incr;
            //g_ptr->theta2 = (btree_prc_raw(N, r, style, T - incr, F, S_o, q, sigma, nullptr) - f0) / incr;
        }

        delete[] nodes;
        return f0;
    }


    double Option::MC_prc(int N_sims, int N_steps, double r, bool get_greeks, bool antithetic, double& sd_prc) {
        Greeks* g_ptr{ get_greeks ? &_greeks : nullptr };
        _prc = MC_prc_terms_asset(_terms, _asset, N_sims, N_steps, r, g_ptr, antithetic, sd_prc);
        return _prc;
    }

    double MC_prc_opt(Option& opt, int N_sims, int N_steps, double r, bool greeks, bool antithetic, double& sd_prc) {
        return opt.MC_prc(N_sims, N_steps, r, greeks, antithetic, sd_prc);
    }

    double MC_prc_terms_asset(const Terms& t, const Asset& a, int N_sims, int N_steps, double r,
        Option::Greeks* g_ptr, bool antithetic, double& sd_prc) {
        if (t.style() == Terms::Style::Amer) {
            throw std::invalid_argument("In MC_prc_terms_asset, Can't price Amer option via MC.");
        }
        return MC_prc_raw(N_sims, N_steps, r, t.style(), t.T(), t.payoff_fn(), t.path_payoff(),
            t.path_accum(), t.init_accum(), a.S_o(), a.q(), a.sigma(), g_ptr, antithetic, sd_prc);
    }

    double MC_prc_raw(int N_sims, int N_steps, double r, Terms::Style style, double T, std::function<double(double)> payoff,
        std::function<double(double, double)> path_payoff, std::function<void(int, double, double&)> path_accum_fn, double init_accum,
        double S_o, double q, double sigma, Option::Greeks* g_ptr, bool antithetic, double& sd_mean_PV) {

        if (style == Terms::Style::Amer) {
            throw std::invalid_argument("In MC_prc_raw, no Amer terms allowed. \n");
        }

        double g = r - q - sigma * sigma / 2;  // risk neutral growth rate of lnS.
        double df = exp(-r * T);

        // long unsigned int seed = 1;    // Replace next line with these 2 for reproducing random 
        // static std::mt19937 gen{seed}; // bit streams for testing.
        static std::mt19937 gen{ (std::random_device{})() };
        std::normal_distribution<> norm{ };

        double cumm_P{ 0.0 };
        double ssq{ 0.0 };  // track ssq for Var[P] estimate.
        if (style == Terms::Style::Euro) {
            // Can by-pass path stepping and just simulate final risk-neutral asset price at time T:
            double sd = sigma * sqrt(T);

            if (!g_ptr) {  // nullptr, skip greeks:
                if (!antithetic) {
                    for (int i = 0; i < N_sims; ++i) {
                        double eps = norm(gen);
                        double P = payoff(S_o * exp(g * T + eps * sd));
                        cumm_P += P;
                        ssq += P * P; 
                    }
                } else {  // antithetic: avg +eps result with -eps result on each sim, i:
                    for (int i = 0; i < N_sims; ++i) {
                        double eps = norm(gen);
                        double P = (payoff(S_o * exp(g * T + eps * sd)) +
                            payoff(S_o * exp(g * T - eps * sd))) / 2;
                        cumm_P += P;
                        ssq += P * P;
                    }
                }
            } else {      // valid g_ptr, get greeks:
                double incr = 0.01;  // Fractional increment to underlying variable for each greek.
                double S_o_up{ (1. + incr) * S_o }, S_o_dn{ (1. - incr) * S_o }, g_rho_up{ g + incr * r };
                double cumm_delta_up{ 0 }, cumm_delta_dn{ 0 }, cumm_rho_up{ 0 }, cumm_theta_up{ 0 }, cumm_vega_up{ 0 };
                double sigma_up{ (1 + incr) * sigma };
                if (!antithetic) {
                    for (int i = 0; i < N_sims; ++i) {
                        double eps = norm(gen);
                        double P = payoff(S_o * exp(g * T + eps * sd));
                        cumm_P += P;
                        ssq += P * P;
                        // and now accumulate for the greeks:
                        cumm_delta_up += payoff(S_o_up * exp(g * T + eps * sd));
                        cumm_delta_dn += payoff(S_o_dn * exp(g * T + eps * sd));
                        cumm_rho_up += payoff(S_o * exp(g_rho_up * T + eps * sd));
                        cumm_theta_up += payoff(S_o * exp(g * (1 - incr) * T + eps * sigma * sqrt((1 - incr) * T)));
                        cumm_vega_up += payoff(S_o * exp((r - q - sigma_up * sigma_up / 2) * T + eps * sigma_up * sqrt(T)));
                    }
                } else {
                    for (int i = 0; i < N_sims; ++i) {
                        double eps = norm(gen);
                        double P = (payoff(S_o * exp(g * T + eps * sd)) +
                            payoff(S_o * exp(g * T - eps * sd))) / 2;
                        cumm_P += P;
                        ssq += P * P;
                        cumm_delta_up += (payoff(S_o_up * exp(g * T + eps * sd)) +
                            payoff(S_o_up * exp(g * T - eps * sd))) / 2;
                        cumm_delta_dn += (payoff(S_o_dn * exp(g * T + eps * sd)) +
                            payoff(S_o_dn * exp(g * T - eps * sd))) / 2;
                        cumm_rho_up += (payoff(S_o * exp(g_rho_up * 
                        T + eps * sd)) +
                            payoff(S_o * exp(g_rho_up * T - eps * sd))) / 2;
                        cumm_theta_up += (payoff(S_o * exp(g * (1 - incr) * T + eps * sigma * sqrt((1 - incr) * T))) +
                            payoff(S_o * exp(g * (1 - incr) * T - eps * sigma * sqrt((1 - incr) * T)))) / 2;
                        cumm_vega_up += (payoff(S_o * exp((r - q - sigma_up * sigma_up / 2) * T + eps * sigma_up * sqrt(T))) +
                            payoff(S_o * exp((r - q - sigma_up * sigma_up / 2) * T - eps * sigma_up * sqrt(T)))) / 2;
                    }
                }
                /* By linearity these greeks have been calculated once in terms of the accumulated
                   values. To estimate their stddevs, as we do for price, they would need to be moved inside
                   the loop over N_sims to and the squares of their values accumulated. The code as 
                   written is much more compact/readable...
                   */
                double delta_up = df * (cumm_delta_up - cumm_P) / (N_sims * incr * S_o);
                double delta_dn = df * (cumm_P - cumm_delta_dn) / (N_sims * incr * S_o);
                g_ptr->delta = (delta_dn + delta_up) / 2.0;
                g_ptr->gamma = (delta_up - delta_dn) / (incr * S_o);
                g_ptr->rho = df * (exp(-incr * r * T) * cumm_rho_up - cumm_P) / (N_sims * incr * r);
                g_ptr->theta = df * (exp(r * incr * T) * cumm_theta_up - cumm_P) / (N_sims * incr * T);
                g_ptr->vega = df * (cumm_vega_up - cumm_P) / (N_sims * incr * sigma);
            }
        } else { // Asian, like Euro but must simulate each time step:
            double dt = T / N_steps;
            double sd = sigma * sqrt(dt);
            double path_mean;  // Asians track path means of price, and optionally greeks.
            double S;
            if (!g_ptr) { // nullptr -skip greeks
                if (!antithetic) {
                    for (int i = 0; i < N_sims; ++i) {
                        S = S_o;
                        path_mean = init_accum;
                        for (int j = 0; j < N_steps; ++j) {
                            double eps = norm(gen);
                            S *= exp(g * dt + eps * sd);
                            path_accum_fn(j + 1, S, path_mean);  // Could use (S_prev + S) / 2.
                        }
                        double P = (style == Terms::Style::AsianPrc) ? payoff(path_mean) : path_payoff(path_mean, S);
                        cumm_P += P;
                        ssq += P * P;
                    }
                } else { // Perform antithetic sampling:
                    // Introduce the antithetic path 'accumulators':
                    double path_mean_anti;
                    double S_anti;
                    for (int i = 1; i <= N_sims; ++i) {
                        // Initialize path accumulators prior to this(i) sim/run:
                        path_mean = init_accum;
                        path_mean_anti = init_accum;
                        S = S_o;
                        S_anti = S_o;
                        for (int j = 0; j < N_steps; ++j) {
                            double eps = norm(gen);
                            S *= exp(g * dt + eps * sd);
                            path_accum_fn(j + 1, S, path_mean);
                            S_anti *= exp(g * dt - eps * sd);
                            path_accum_fn(j + 1, S_anti, path_mean_anti);
                        }
                        double P = (style == Terms::Style::AsianPrc) ? (payoff(path_mean) + payoff(path_mean_anti)) / 2
                            : (path_payoff(path_mean, S) + path_payoff(path_mean_anti, S_anti)) / 2;
                        cumm_P += P;
                        ssq += P * P;
                    }
                }
            } else { // valid g_ptr, get greeks:
                double incr = 0.01;
                double sigma_up{ (1 + incr) * sigma };
                // Introduce and Initialize new accumulators(needed for the greeks) before the N_sims:
                double cumm_delta_up{ 0 }, cumm_delta_dn{ 0 }, cumm_rho_up{ 0 },
                    cumm_theta_up{ 0 }, cumm_vega_up{ 0 };
                // S, path_mean and the following 8 path accumulators get reset for each of the N_sims by init_sim[&]():
                double S_rho_up, S_theta_up, S_vega_up;
                double path_mean_up, path_mean_dn;
                double pm_rho_up, pm_vega_up, pm_theta_up;
                if (!antithetic) {
                    std::function<void()> init_sim{ [&] () {
                        S = S_o; S_rho_up = S_o; S_theta_up = S_o; S_vega_up = S_o;
                        path_mean = init_accum; path_mean_up = init_accum; path_mean_dn = init_accum;
                        pm_rho_up = init_accum; pm_vega_up = init_accum;
                    } };
                    for (int i = 0; i < N_sims; ++i) {
                        init_sim();  // reset this sim run accumulators.
                        for (int j = 0; j < N_steps; ++j) {
                            double eps = norm(gen);
                            S *= exp(g * dt + eps * sd);
                            path_accum_fn(j + 1, S, path_mean);
                            path_accum_fn(j + 1, S * (1 + incr), path_mean_up);
                            path_accum_fn(j + 1, S * (1 - incr), path_mean_dn);
                            S_rho_up *= exp((g + incr * r) * dt + eps * sd);
                            path_accum_fn(j + 1, S_rho_up, pm_rho_up);
                            S_vega_up *= exp((r - q - sigma_up * sigma_up / 2) * dt + eps * sigma_up * sqrt(dt));
                            path_accum_fn(j + 1, S_vega_up, pm_vega_up);
                            if (j == N_steps - 2) { // special treatment -conditional has execution cost time(could put outside loop).
                                pm_theta_up = path_mean;
                                S_theta_up = S;
                            }
                        }
                        double P{};
                        if (style == Terms::Style::AsianPrc) {
                            P = payoff(path_mean);
                            cumm_delta_up += payoff(path_mean_up);
                            cumm_delta_dn += payoff(path_mean_dn);
                            cumm_rho_up += payoff(pm_rho_up);
                            cumm_vega_up += payoff(pm_vega_up);
                            cumm_theta_up += payoff(pm_theta_up);
                        } else {  // AsianStrike and AsianExotic payoff depends on path_means and S(T):
                            P = path_payoff(path_mean, S);
                            cumm_delta_up += path_payoff(path_mean_up, S * (1 + incr));
                            cumm_delta_dn += path_payoff(path_mean_dn, S * (1 - incr));
                            cumm_rho_up += path_payoff(pm_rho_up, S);
                            cumm_vega_up += path_payoff(pm_vega_up, S);
                            cumm_theta_up += path_payoff(pm_theta_up, S_theta_up);
                        }
                        cumm_P += P;
                        ssq += P * P;
                    }
                } else {  // antithetic:
                    // Introduce the antithetic path 'accumulators':
                    double path_mean_anti;
                    double S_anti;
                    double S_rho_up_anti, S_theta_up_anti, S_vega_up_anti;
                    double path_mean_up_anti, path_mean_dn_anti;
                    double pm_rho_up_anti, pm_vega_up_anti, pm_theta_up_anti;

                    // init_sim_antith() called to initialize path accumulators prior to each of the N_sims:
                    std::function<void()> init_sim_antith{ [&] () {
                        S = S_o; S_rho_up = S_o; S_theta_up = S_o; S_vega_up = S_o;
                        path_mean = init_accum; path_mean_up = init_accum; path_mean_dn = init_accum;
                        pm_rho_up = init_accum; pm_vega_up = init_accum;
                        S_anti = S_o; S_rho_up_anti = S_o; S_theta_up_anti = S_o; S_vega_up_anti = S_o;
                        path_mean_anti = init_accum; path_mean_up_anti = init_accum; path_mean_dn_anti = init_accum;
                        pm_rho_up_anti = init_accum; pm_vega_up_anti = init_accum;
                    } };

                    for (int i = 0; i < N_sims; ++i) {
                        init_sim_antith();
                        for (int j = 0; j < N_steps; ++j) {
                            double eps = norm(gen);
                            S *= exp(g * dt + eps * sd);
                            path_accum_fn(j + 1, S, path_mean);
                            path_accum_fn(j + 1, S * (1 + incr), path_mean_up);
                            path_accum_fn(j + 1, S * (1 - incr), path_mean_dn);
                            S_rho_up *= exp((g + incr * r) * dt + eps * sd);
                            path_accum_fn(j + 1, S_rho_up, pm_rho_up);
                            S_vega_up *= exp((r - q - sigma_up * sigma_up / 2) * dt + eps * sigma_up * sqrt(dt));
                            path_accum_fn(j + 1, S_vega_up, pm_vega_up);
                            if (j == N_steps - 2) { // special treatment -conditional has execution cost time(could put outside loop).
                                pm_theta_up = path_mean;
                                S_theta_up = S;
                            }
                            // Same for antithetic path, just replace +eps with -eps:
                            S_anti *= exp(g * dt - eps * sd);
                            path_accum_fn(j + 1, S_anti, path_mean_anti);
                            path_accum_fn(j + 1, S_anti * (1 + incr), path_mean_up_anti);
                            path_accum_fn(j + 1, S_anti * (1 - incr), path_mean_dn_anti);
                            S_rho_up_anti *= exp((g + incr * r) * dt - eps * sd);
                            path_accum_fn(j + 1, S_rho_up_anti, pm_rho_up_anti);
                            S_vega_up_anti *= exp((r - q - sigma_up * sigma_up / 2) * dt - eps * sigma_up * sqrt(dt));
                            path_accum_fn(j + 1, S_vega_up_anti, pm_vega_up_anti);
                            if (j == N_steps - 2) {
                                pm_theta_up_anti = path_mean_anti;
                                S_theta_up_anti = S_anti;
                            }
                        }
                        // Accumulate for this(i) sim/run:
                        double P{};
                        if (style == Terms::Style::AsianPrc) {
                            P = (payoff(path_mean) + payoff(path_mean_anti)) / 2;  // averaging antithetic path results.
                            cumm_delta_up += (payoff(path_mean_up) + payoff(path_mean_up_anti)) / 2;
                            cumm_delta_dn += (payoff(path_mean_dn) + payoff(path_mean_dn_anti)) / 2;
                            cumm_rho_up += (payoff(pm_rho_up) + payoff(pm_rho_up_anti)) / 2;
                            cumm_vega_up += (payoff(pm_vega_up) + payoff(pm_vega_up_anti)) / 2;
                            cumm_theta_up += (payoff(pm_theta_up) + payoff(pm_theta_up_anti)) / 2;
                        } else {
                            P = (path_payoff(path_mean, S) + path_payoff(path_mean_anti, S_anti)) / 2;
                            cumm_delta_up += (path_payoff(path_mean_up, S * (1 + incr)) + path_payoff(path_mean_up_anti, S_anti * (1 + incr))) / 2;
                            cumm_delta_dn += (path_payoff(path_mean_dn, S * (1 - incr)) + path_payoff(path_mean_dn_anti, S_anti * (1 - incr))) / 2;
                            cumm_rho_up += (path_payoff(pm_rho_up, S_rho_up) + path_payoff(pm_rho_up_anti, S_rho_up_anti)) / 2;
                            cumm_vega_up += (path_payoff(pm_vega_up, S_vega_up) + path_payoff(pm_vega_up_anti, S_vega_up_anti)) / 2;
                            cumm_theta_up += (path_payoff(pm_theta_up, S_theta_up) + path_payoff(pm_theta_up_anti, S_theta_up_anti)) / 2;
                        }
                        cumm_P += P;
                        ssq += P * P;
                    }

                }
                double delta_up = df * (cumm_delta_up - cumm_P) / (N_sims * incr * S_o);
                double delta_dn = df * (cumm_P - cumm_delta_dn) / (N_sims * incr * S_o);
                g_ptr->delta = (delta_dn + delta_up) / 2.0;
                g_ptr->gamma = (delta_up - delta_dn) / (incr * S_o);
                g_ptr->rho = df * (exp(-incr * r * T) * cumm_rho_up - cumm_P) / (N_sims * incr * r);
                g_ptr->theta = df * (exp(r * dt) * cumm_theta_up - cumm_P) / (N_sims * dt); // uses dt, not incr*T.
                g_ptr->vega = df * (cumm_vega_up - cumm_P) / (N_sims * incr * sigma);
            }
        }

        double mean_P = cumm_P / N_sims;
        sd_mean_PV = df * sqrt( ( (ssq - N_sims*mean_P*mean_P) / (N_sims-1)) / N_sims);
        double mean_PV = df * mean_P;

        /* // These had much higher variance than by using same sample paths for up/down estimates..
                if (g_ptr) {
                    double incr = 0.01;
                    double delta_up = (MC_prc_raw(N_sims, N_steps, r, style, T, payoff, path_payoff, path_accum_fn,
                        (1 + incr) * S_o, q, sigma, nullptr, antithetic, sd_mean_P) - mean_P) / (incr * S_o);
                    double delta_down = (mean_P - MC_prc_raw(N_sims, N_steps, r, style, T, payoff, path_payoff, path_accum_fn,
                        (1 - incr) * S_o, q, sigma, nullptr, antithetic, sd_mean_P)) / (incr * S_o);
                    g_ptr->delta = (delta_down + delta_up) / 2.0;
                    g_ptr->gamma = (delta_up - delta_down) / (incr * S_o);
                    g_ptr->rho = (MC_prc_raw(N_sims, N_steps, (1 + incr) * r, style, T, payoff, path_payoff, path_accum_fn,
                        S_o, q, sigma, nullptr, antithetic, sd_mean_P) - mean_P) / (incr * r);
                    g_ptr->vega = (MC_prc_raw(N_sims, N_steps, r, style, T, payoff, path_payoff, path_accum_fn,
                        S_o, q, (1 + incr) * sigma, nullptr, antithetic, sd_mean_P) - mean_P) / (incr * sigma);
                    g_ptr->theta = (MC_prc_raw(N_sims, N_steps, r, style, (1 - incr) * T, payoff, path_payoff, path_accum_fn,
                        S_o, q, sigma, nullptr, antithetic, sd_mean_P) - mean_P) / (incr * T);
                }
        */
        return mean_PV;
    }

    std::ostream& operator<< (std::ostream& os, const Terms& t) {
        switch (t.style()) {
        case Terms::Style::Euro: os << "European ";  break;
        case Terms::Style::Amer: os << "American ";  break;
        case Terms::Style::AsianPrc: os << "AsianPrc ";  break;
        case Terms::Style::AsianStrike: os << "AsianStrike ";  break;
        case Terms::Style::AsianExotic: os << "AsianExotic ";  break;
            // case Terms::Style::Other: os << "Other "; break;
        default: os.setstate(std::ios_base::failbit);
        }
        if (t.style() == Terms::Style::AsianPrc || t.style() == Terms::Style::AsianStrike
            || t.style() == Terms::Style::AsianExotic) {
            switch (t.avgType()) {
            case Terms::AvgType::Arith: os << "Arithmetic ";  break;
            case Terms::AvgType::Geom: os << "Geometric ";  break;
            case Terms::AvgType::Other: os << "Other ";  break;
            case Terms::AvgType::None: os << "None ";  break;
            default: os.setstate(std::ios_base::failbit);
            }
        }


        switch (t.type()) {
        case Terms::Type::Call: os << "Call  ";  break;
        case Terms::Type::Put: os << "Put   ";  break;
        case Terms::Type::Other: os << "Other   "; break;
        default: os.setstate(std::ios_base::failbit);
        }
        switch (t.style()) {
        case Terms::Style::Euro:
        case Terms::Style::Amer:
        case Terms::Style::AsianPrc: os << t.K();  break;
        default: os << '-';
        }
        os << ' ' << t.T();
        return os;
    }

    std::ostream& operator<< (std::ostream& os, const Asset& a) {
        os << a.S_o() << "  " << a.q() << "  " << a.sigma();
        return os;
    }

    std::ostream& operator<< (std::ostream& os, const Option& opt) {
        os << opt.terms() << ' ' << opt.asset();
        return os;
    }
}


### An Implementation
Borrowing from ML terminology we will calculate the optimal $c^*$ on the training set of binomial trees having `N_steps` time steps on [20, 100], and then we will use this $c^*$ to estimate the price of the American option on a large tree having `N_steps` near 1000. We will first confirm that this choice of $c^*$ leads to reduced variance in line with the above formula.
The optimal $c$ may depend on the option parameters, so it is best to estimate $c^*$ with the particular parameter set of interest.

Then we calculate the sample variances for each of: the American option prices, the American option prices using control variate with $c=1$, the American option prices using the optimal control variate ($c^*$). This confirms that use of $c^*$ reduces variance by the factor $(1 - Corr^2[X, Y]) = 1 - 0.990989^2=0.01794$.

Repeating the same calculations for a `test` set of binomial trees having `N_steps` on [1000, 1080] confirms that the $c^*$ obtained from the training set does a good job reducing the variance. Of course $c^*_large$ obtained from the test set does a better job (and that should therefore be used if you are generating enough large trees to give a good estimate of $c^*$. )

Note: The variance  is small enough to see a trend: the estimated American option prices decrease from 1.99065 to 1.99055 as `N_steps` increases from 1000 to 4000. Increasing `N_steps` by 1 near `N_steps`=4000 shows the familiar up/down price dependence. The up/down price envelope of the covariate corrected prices is decreasing, the (larger) up/down price envelope of the raw prices is increasing. Exrapolating these, or fitting some curve to them, may give a better option price estimate.

The upshot of all this, barring completion of the mentioned study to fit extrapolating curves, is that a European option as control variate with $c^* = c^*_{small}$ gives low variance price estimates for the corresponding American. But one should sample prices from a few binomial trees having larger `N_steps` to see if there is a trend detectable given this low noise observational tool.

In [3]:
// control_var.cc:
using namespace OptionPricing;

//using Terms::Style::Amer, Terms::Type::Put;
using std::cout;
using std::endl;
using std::valarray;

// Construct the Amer and Euro Put options for a particular parameter set:
double S_o = 40.0, r = 0.0488, q = 0.0;
double K = 40.0, T = 7 / 12., sigma = 0.2;
Asset underlying{ S_o, q, sigma };
Terms terms_amer_put{ Terms::Style::Amer, Terms::Type::Put, T, K };
Terms terms_euro_put{ Terms::Style::Euro, Terms::Type::Put, T, K };
Option amer_put(terms_amer_put, underlying);
Option euro_put(terms_euro_put, underlying);

// Get their prices on the training set of binomial trees having N_steps on [20, 100] into
// the arrays amer_prcs, euro_prcs:
bool get_greeks = false;
constexpr int start_N = 20, end_N = 100;
constexpr int N_points{ end_N - start_N + 1 }; // Number of training points.

valarray<double> amer_prcs(N_points);
valarray<double> euro_prcs(N_points);


for (int N_steps = start_N; N_steps < end_N + 1; ++N_steps) {
    amer_prcs[N_steps - start_N] = amer_put.btree_prc(N_steps, r, get_greeks);
    euro_prcs[N_steps - start_N] = euro_put.btree_prc(N_steps, r, get_greeks);
}

// Calculate their sample means, vars and covariance:
double amer_mean = amer_prcs.sum() / N_points;
double euro_mean = euro_prcs.sum() / N_points;

double var_amer = ((amer_prcs - amer_mean) * (amer_prcs - amer_mean)).sum() / (N_points - 1);
double var_euro = ((euro_prcs - euro_mean) * (euro_prcs - euro_mean)).sum() / (N_points - 1);
double cov_a_e = ((amer_prcs - amer_mean) * (euro_prcs - euro_mean)).sum() / (N_points - 1);


// Get the theoretical optimum coefficient, c (for this parameter set):
double c = cov_a_e / var_euro;
cout << "Training Set (binomial trees having N_steps on [" << start_N << " : "
<< end_N << "]) Results:" << endl;
cout << "Optimal c: " << c << endl;
cout << "Corr(Amer, Euro): " << cov_a_e / (sqrt(var_amer) * sqrt(var_euro)) << endl;

// Get the Black-Scholes Euro option price:
double d_1 = (std::log(S_o / K) + (r - q + sigma * sigma / 2) * T) / (sigma * std::sqrt(T));
double d_2 = d_1 - sigma * std::sqrt(T);
double BS_prc = std::exp(-r * T) * K * std::erfc(d_2 / std::sqrt(2)) / 2
- S_o * std::erfc(d_1 / std::sqrt(2)) / 2;

// Get the variance of Amer option prices via the optimal, and the Hull, White (c==1) control
// variate admixture:
valarray<double> amer_prcs_HW = amer_prcs - (euro_prcs - BS_prc);
valarray<double> amer_prcs_opt = amer_prcs - c * (euro_prcs - BS_prc);
double amer_mean_HW = amer_prcs_HW.sum() / N_points;
double amer_mean_opt = amer_prcs_opt.sum() / N_points;
double var_amer_HW = ((amer_prcs_HW - amer_mean_HW) * (amer_prcs_HW - amer_mean_HW)).sum() / (N_points - 1);
double var_amer_opt = ((amer_prcs_opt - amer_mean_opt) * (amer_prcs_opt - amer_mean_opt)).sum() / (N_points - 1);
cout << "         c                  Var | stddev" << endl;
cout << "None                 " << var_amer << '|' << sqrt(var_amer) << endl;
cout << "Hull-White, c=1      " << var_amer_HW << '|' << sqrt(var_amer_HW) << endl;
cout << "Optimal c=Cov/Var    " << var_amer_opt << '|' << sqrt(var_amer_opt) << endl;


// Out of sample test on binomial trees same number, N_points, of points as training sample
// but all trees having N_steps >> end_N.
// Test grid: [1000 : 1000 + N_points]
int test_start_N = 1000;
int test_end_N = 1000 + N_points - 1;
cout << "\nTest set (binomial trees having N_steps on [" << test_start_N << " : "
<< test_end_N << "]) Results:" << endl;

for (int N_steps = test_start_N; N_steps < test_end_N + 1; ++N_steps) {
    amer_prcs[N_steps - test_start_N] = amer_put.btree_prc(N_steps, r, get_greeks);
    euro_prcs[N_steps - test_start_N] = euro_put.btree_prc(N_steps, r, get_greeks);
}

// Calculate their sample means, vars and covariance:
amer_mean = amer_prcs.sum() / N_points;
euro_mean = euro_prcs.sum() / N_points;

var_amer = ((amer_prcs - amer_mean) * (amer_prcs - amer_mean)).sum() / (N_points - 1);
var_euro = ((euro_prcs - euro_mean) * (euro_prcs - euro_mean)).sum() / (N_points - 1);
cov_a_e = ((amer_prcs - amer_mean) * (euro_prcs - euro_mean)).sum() / (N_points - 1);
// Get the theoretical optimum mix of the Euro option control variate (for this parameter set):
double c_testset = cov_a_e / var_euro;
cout << "Optimal c (in test set): " << c_testset << endl;
cout << "Test set Corr(Amer, Euro): " << cov_a_e / (sqrt(var_amer) * sqrt(var_euro)) << endl;

// Get the variance of Amer option prices via the optimal, and the Hull, White (c==1) control
// variate admixture:
amer_prcs_HW = amer_prcs - (euro_prcs - BS_prc);
amer_prcs_opt = amer_prcs - c * (euro_prcs - BS_prc); // Use c from training set!
amer_mean_HW = amer_prcs_HW.sum() / N_points;
amer_mean_opt = amer_prcs_opt.sum() / N_points;
var_amer_HW = ((amer_prcs_HW - amer_mean_HW) * (amer_prcs_HW - amer_mean_HW)).sum() / (N_points - 1);
var_amer_opt = ((amer_prcs_opt - amer_mean_opt) * (amer_prcs_opt - amer_mean_opt)).sum() / (N_points - 1);

cout << "         c                  Var | stddev" << endl;
cout << "None                 " << var_amer << '|' << sqrt(var_amer) << endl;
cout << "Hull-White, c=1      " << var_amer_HW << '|' << sqrt(var_amer_HW) << endl;
cout << "Optimal c=Cov/Var    " << var_amer_opt << '|' << sqrt(var_amer_opt) << endl;


Training Set (binomial trees having N_steps on [20 : 100]) Results:
Optimal c: 0.669685
Corr(Amer, Euro): 0.990989
         c                  Var | stddev
None                 8.17512e-05|0.00904164
Hull-White, c=1      2.09987e-05|0.00458243
Optimal c=Cov/Var    1.46672e-06|0.00121108

Test set (binomial trees having N_steps on [1000 : 1080]) Results:
Optimal c (in test set): 0.676637
Test set Corr(Amer, Euro): 0.99997
         c                  Var | stddev
None                 1.48754e-07|0.000385687
Hull-White, c=1      3.39803e-08|0.000184337
Optimal c=Cov/Var    2.46169e-11|4.96154e-06
